In [1]:
import json
from time import sleep

import pyautogui as pg
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
def selenium_run():
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)
    url = 'https://code4biz.slack.com/'

    # 暗黙的な待機
    driver.implicitly_wait(5)
    driver.get(url)
    driver.maximize_window()
    return driver

In [9]:
def auto_login(driver):
    # ログインページ処理
    form = driver.find_element(by=By.CSS_SELECTOR, value='#signin_form')
    login_mail = form.find_element(by=By.NAME, value='email')
    login_passwd = form.find_element(by=By.NAME, value='password')

    # 認証情報
    with open('src/auth.json') as f:
        auth = json.load(f)

    my_mail = auth['my_mail']
    my_pass = auth['my_pass']

    # ログイン情報初期化
    login_mail.clear()
    login_passwd.clear()

    login_mail.send_keys(my_mail)
    login_passwd.send_keys(my_pass)

    # サインインボタンを押す
    sleep(0.5)
    btn = form.find_element(by=By.CSS_SELECTOR, value='#signin_btn')
    btn.click()
    sleep(2)

    # Slackアプリ起動確認メッセージのキャンセル
    pg.press('enter', presses=1, interval=0.1)
    sleep(1)

    # ブラウザでSlack起動
    pg.press('tab', presses=2, interval=0.1)
    pg.press('enter', presses=1, interval=0.1)
    sleep(2)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\sgwya\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [101]:
def to_search_yesterday():
    # 検索フィルターで期間指定
    pg.press('tab', presses=12, interval=0.1)
    pg.press('enter', presses=1, interval=0.1)
    sleep(1)

    # 日付検索を昨日にする
    pg.press('tab', presses=4, interval=0.1)
    pg.press('enter', presses=1, interval=0.1)
    pg.press('down', presses=3, interval=0.1)
    pg.press('enter', presses=1, interval=0.1)


    pg.press('tab', presses=5, interval=0.1)
    pg.press('enter', presses=1, interval=0.1)

    # 新→旧表示
    # pg.press('tab', presses=15, interval=0.1)
    # pg.press('enter', presses=1, interval=0.1)
    # pg.press('down', presses=1, interval=0.1)
    # pg.press('enter', presses=1, interval=0.1)

    # 100件表示
    # pg.press('tab', presses=16, interval=0.1)
    # pg.press('enter', presses=1, interval=0.1)
    # pg.press('down', presses=2, interval=0.1)
    # pg.press('enter', presses=1, interval=0.1)


In [102]:
driver = selenium_run()
auto_login(driver)

# SlackのWebページが完全に立ち上がるまで5秒待機
sleep(8)
to_search_yesterday()
sleep(1)

message_groups = driver.find_elements(by=By.CLASS_NAME, value='c-message_group')
# len(message_groups)
message = message_groups[4]
date = message.find_element(by=By.CLASS_NAME, value='c-message_group__header_date').text
timestamp = message.find_element(by=By.CLASS_NAME, value='c-timestamp__label').text
channel_name = message.find_element(by=By.CLASS_NAME, value='p-deprecated_channel_name__text').text
sender_name = message.find_element(by=By.CLASS_NAME, value='c-message__sender').text
text_section = message.find_element(by=By.CLASS_NAME, value='p-rich_text_section').text
url = message.find_elements(by=By.TAG_NAME, value='a')[-1].get_attribute('href')
datum = {
    '投稿日' : date,
    '投稿時間' : timestamp,
    '投稿チャンネル' : channel_name,
    '投稿者' : sender_name,
    '投稿メッセージ' : text_section,
    'リンク' : url
}
datum

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\sgwya\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


{'投稿日': '6月30日',
 '投稿時間': '21:19',
 '投稿チャンネル': '01-いまにゅのつぶやき',
 '投稿者': '片山真介',
 '投稿メッセージ': '@Imanyu\n\n「code4biz 受講後、もう１ランクアップするために必要なことと乗り越え方」\ncode4biz 第一期の受講が終わっても、Python の学習は絶対に続けよう！と思えるようになりました\n... もっと表示する',
 'リンク': 'https://code4biz.slack.com/team/U02HDQ4GGUE'}

'ありがとうございます！'